In [75]:
from bs4 import BeautifulSoup
import requests
from lxml import html
import urllib.request
import re

pages = []
hrefs = []

for i in range(1, 3985):
    url = 'https://vv-34.ru/all/page-'+str(i)
    pages.append(url)
    
with open('urls.txt', 'a') as urls:
    for each in pages:
        f = urllib.request.urlopen(each)
        file = f.read()
        soup = BeautifulSoup(file)
        newentries = soup.find_all('div', {'class': 'newentry'})
        rightcats = soup.find_all('div', {'class': 'rightcat'})
        for eachh in rightcats:
            a = eachh.find_all('a', href=True)[0]
            new = 'https://vv-34.ru' + str(a.get('href'))[:-2]
            urls.write(new+'\n')


In [178]:
from bs4 import BeautifulSoup
import requests
from lxml import html
import urllib.request
import re
from lxml import etree


with open('urls.txt') as all_urls:
    urli = all_urls.read().split()
    def scrapper():
        for url in urli:
            stranica = []
            full_text = []
            stranica.append('######')
            stranica.append('URL: ' + url)
            stranica.append('Источник: Вечерний Волгоград')
            f = urllib.request.urlopen(url)
            file = f.read()
            soup = BeautifulSoup(file)
            con_pubdate = soup.find('div', {'class': 'con_pubdate'})
            try:
                con_heading = soup.find('h1', {'class': 'con_heading'})
                heading = 'Название статьи: ' + con_heading.text
            except AttributeError:
                heading = 'Название статьи не указано'
            pattern = '\n|\t|\s'
            dat = con_pubdate.text
            dt = re.sub(pattern, '', dat)
            months = {'января': '.01.', 'февраля': '.02.', 'марта': '.03.', 'апреля': '.04.', 'мая': '.05.', 'июня': '.06.', 'июля': '.07.', 'августа': '.08.', 'сентября': '.09.', 'октября': '.10.', 'ноября': '.11.', 'декабря': '.12.'}
            pat = '[а-я]+'
            dtt = re.findall(pat, dt)
            d = re.sub(dtt[0], months[dtt[0]], dt)          
            date = 'Дата: ' + d
            stranica.append(date)
            stranica.append(heading)
            artconts = soup.find_all('div', {'class': 'artcont'})
            for each in artconts:     
                try:
                    em = each.find_all('em')[0]
                    author = 'Автор: '+ em.text
                    stranica.append(author)
                except IndexError:
                    stranica.append('Автор не указан')
                try:
                    for i in range(len(each.find_all('p'))):
                        al = each.find_all('p')[i]
                        #print(al)
                        full_text.append(al.text)
                except IndexError:
                    pass
                #print(full_text)
                for i in range(len(each.find_all('span', attrs={'style': 'font-size:16px;'}))):
                    span = each.find_all('span', attrs={'style': 'font-size:16px;'})[i]
                    if span.text not in stranica:
                        a = span.text.split('\n')
                        full_text.append(a[0])
            textt = ' '.join(full_text)
            pttrn = '\n|\t'
            tex = re.sub(pttrn, '', textt)
            stranica.append(tex)
            strnc = '\n'.join(stranica)
            with open ('data.txt', 'a', encoding='utf-8') as data:
                data.write(strnc + '\n')
                
    while True:
        try:
            scrapper()
            break
        except TimeoutError:
            sleep(10)
            print('ops')
            pass